In [33]:
#libraries
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from matplotlib import pyplot
eps = np.finfo(float).eps
from numpy import log2 as log
# Quick value count calculator
from collections import Counter

In [34]:

#a function for read data set
df = pd.read_csv("/Users/najmeh/Desktop/master/Ml/projet/glass.csv")
features = df.columns[:-1].tolist()
print(df.shape)
df.head(2)

#transform data six class 1,2,3,5,6,7 in target
pd.cut(df['Type'], bins=6).value_counts()
df['Type'].plot(kind='hist')

#spilit data to two group(training and test)

training_data = df.sample(frac=0.7, random_state=25)
testing_data = df.drop(training_data.index)

print(f"number of training examples: {training_data.shape[0]}")
print(f"number of testing examples: {testing_data.shape[0]}")

(214, 10)
number of training examples: 150
number of testing examples: 64


In [35]:
#calculate entropy
entropy_node = 0  
#Unique objects - 1,2,3,5,6,,7
#here attribute = 'Type'
values = df.Type.unique()  
for value in values:
    fraction = df.Type.value_counts()[value]/len(df.Type)  
    entropy_node += -fraction*np.log2(fraction)

In [36]:
#show us = entropy of the whole dataset
entropy_node

2.176533992398201

In [37]:
def ent(df,attribute):
    target_variables = df.Type.unique()  #This gives all 'Yes'=1 and 'No'=0
    variables = df[attribute].unique()    


    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df.Type ==target_variable]) #numerator
            den = len(df[attribute][df[attribute]==variable])  
            fraction = num/(den+eps)  #pi
            entropy_each_feature += -fraction*log(fraction+eps) 
        fraction2 = den/len(df)
        entropy_attribute += -fraction2*entropy_each_feature   #Sums up all the entropy ETaste

    return(abs(entropy_attribute))

In [38]:
#find entropy of all attribute
a_entropy = {k:ent(df,k) for k in df.keys()[:-1]}
a_entropy

{'RI': 0.14371442758020297,
 'Na': 0.45905995774545544,
 'Mg': 0.8454468773975213,
 'Al': 0.6075386377025457,
 'Si': 0.5692221379818928,
 'K': 1.0073159229739324,
 'Ca': 0.4334960559369545,
 'Ba': 1.6170290542672472,
 'Fe': 1.7997372370199376}

In [39]:
#define a function for calculate information gain
def ig(e_dataset,e_attr):
    return(e_dataset-e_attr)
    

In [40]:
IG = {k:ig(entropy_node,a_entropy[k]) for k in a_entropy}
t22=[]

In [41]:
def find_entropy(df):
    Class = df.keys()[-1]  
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  
  
def find_entropy_attribute(df,attribute):
#To make the code generic, changing target variable class name
  Class = df.keys()[-1]  
  #This gives all 'Yes' and 'No'
  target_variables = df[Class].unique()  
  #This gives different features in that attribute (like '0','y' in sex)
  variables = df[attribute].unique()    
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)

#find an attribute that has max ig
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        #Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]
  
#get subtable for a node
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)

# a function for build DT
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   
    
    

    #attribute maximum information gain
    node = find_winner(df)
    
   
    attValue = np.unique(df[node])
    
    #create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Type'],return_counts=True)      

        #Checking purity of subset that purity shoould be 0
        if len(counts)==1:
            tree[node][value] = clValue[0]
            
        else:        
            #Calling the function recursively 
            tree[node][value] = buildTree(subtable) 
                   
    return tree
  
  

## D-Tree 

In [42]:
t=buildTree(df)
t

{'RI': {1.51115: 6,
  1.51131: 7,
  1.51215: 1,
  1.51299: 6,
  1.51316: 5,
  1.51321: 5,
  1.51409: 2,
  1.51508: 7,
  1.51514: {'Na': {14.01: 5, 14.85: 7}},
  1.51531: 7,
  1.51545: 7,
  1.51556: 7,
  1.51567: 1,
  1.51569: 2,
  1.5157100000000001: 1,
  1.5157399999999999: 2,
  1.51588: 2,
  1.51589: 1,
  1.5159: 2,
  1.5159200000000002: 2,
  1.51593: 2,
  1.51594: 2,
  1.51596: {'Na': {12.79: 1, 13.02: 2}},
  1.5160200000000001: 7,
  1.51605: 2,
  1.51609: 7,
  1.5161: 3,
  1.51613: {'Na': {13.88: 7, 13.92: 2}},
  1.51617: 7,
  1.5161799999999999: {'Na': {13.01: 2, 13.53: 1}},
  1.51623: 7,
  1.51625: 2,
  1.51627: 2,
  1.51629: 2,
  1.51631: 2,
  1.5164: {'Na': {12.55: 2, 14.37: 7}},
  1.51643: 3,
  1.51645: {'Na': {13.4: 2, 13.44: 2, 14.94: 7}},
  1.5164600000000001: {'Na': {13.04: 3, 13.41: 2}},
  1.51651: 7,
  1.51652: 2,
  1.51653: 7,
  1.51655: {'Na': {12.75: 2, 13.41: 3}},
  1.5165799999999998: 7,
  1.5166: 2,
  1.51662: 2,
  1.51663: 2,
  1.51665: 3,
  1.5166600000000001: 5,

In [43]:
def pred(inst,tree):
    for nodes in tree.keys():
        
        value=inst[nodes]
        
        tt=tree[nodes][value]
        
        prediction=0
        
        if type(tt) is dict:
            prediction = pred(inst,tt)
        else:
            prediction = tt
            break;
            
    return prediction
    

import pprint
q=dd.shape[0]
acc=0
pp=[]

for i in range(q):
    ss=dd.loc[i, : ]
    prediction = pred(ss,t)
    if prediction != ['Type']:
        acc=acc+1
        pp=np.append(pp,i)
        
pprint.pprint(t)
print('Accuracy = ')
print(1-(acc/dd.shape[0]))
print('missclassified data= ')
pprint.pprint(pp)

{'RI': {1.51115: 6,
        1.51131: 7,
        1.51215: 1,
        1.51299: 6,
        1.51316: 5,
        1.51321: 5,
        1.51409: 2,
        1.51508: 7,
        1.51514: {'Na': {14.01: 5, 14.85: 7}},
        1.51531: 7,
        1.51545: 7,
        1.51556: 7,
        1.51567: 1,
        1.51569: 2,
        1.5157100000000001: 1,
        1.5157399999999999: 2,
        1.51588: 2,
        1.51589: 1,
        1.5159: 2,
        1.5159200000000002: 2,
        1.51593: 2,
        1.51594: 2,
        1.51596: {'Na': {12.79: 1, 13.02: 2}},
        1.5160200000000001: 7,
        1.51605: 2,
        1.51609: 7,
        1.5161: 3,
        1.51613: {'Na': {13.88: 7, 13.92: 2}},
        1.51617: 7,
        1.5161799999999999: {'Na': {13.01: 2, 13.53: 1}},
        1.51623: 7,
        1.51625: 2,
        1.51627: 2,
        1.51629: 2,
        1.51631: 2,
        1.5164: {'Na': {12.55: 2, 14.37: 7}},
        1.51643: 3,
        1.51645: {'Na': {13.4: 2, 13.44: 2, 14.94: 7}},
        1.5164600

/var/folders/x5/1gwz9t2x3wz7f193pgwy6rcc0000gn/T/ipykernel_1590/1759467665.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if prediction != ['Type']:
